# [Module 5.1] Inferencde Pipeline 생성

## Inference Pipeline <a class="anchor" id="pipeline_setup"></a>

inference pipeline 정의: 추론 파이프라인은 데이터에 대해 추론하기 위한 요청을 처리하는 2~5개의 컨테이너로 이루어진 선형 시퀀스로 구성된 Amazon SageMaker 모델입니다.(An inference pipeline is an Amazon SageMaker model that is composed of a linear sequence of two to five containers that process requests for inferences on data.)<br>
[개발자 가이드 정의 :inference pipeline](https://docs.aws.amazon.com/sagemaker/latest/dg/inference-pipelines.html)


아래 그림과 같이 기존 노트북에서 생성한 전처리 모델, Custom PCA 모델 , XGboost 모델의 모델 아티펙트, 학습/추론 docker image 및 환경 변수를 제공하여 1개의 단일 모델인 Inference Pipleline을 생성 합니다. <br>
**입력 데이타 가공이 없이 실제 데이타가 입력이 되면, 1개의 단일 모델(Inference Pipeline)을 통해서 최종적으로 예측 결과인 True, False의 결과 값이 제공 됩니다.**

![Fig.4.1.inference-pipeline-4-models](img/Fig.4.1.inference-pipeline-4-models.png)

(
**Machine Learning Model Pipeline (Inference Pipeline)는 create_model() 를 호출하여 만들 수 있습니다.** <br>
예를 들어 여기서는 the fitted Scikit-learn inference model, custom PCA model, the fitted Xgboost model and the psotprocessing model 의 네가지 모델을 가지고 만듦니다.

In [8]:
import sagemaker
import pandas as pd
import numpy as np
import os
import time
import json
import boto3
from time import strftime, gmtime

In [9]:
role = sagemaker.get_execution_role()

In [10]:
%store -r

In [11]:
timestamp_prefix = strftime("%Y-%m-%d-%H-%M-%S", gmtime())
model_name = 'churn-inference-pipeline-' + timestamp_prefix
client = boto3.client('sagemaker')
response = client.create_model(
    ModelName=model_name,
    Containers=[
        {
            'Image': preprocessor_image_name,
            'ModelDataUrl': preprocessor_model_data,
            'Environment': {
                    "SAGEMAKER_SUBMIT_DIRECTORY": preprocessor_uploaded_code_s3_prefix,
                    "TRANSFORM_MODE": "feature-transform",
                    "SAGEMAKER_PROGRAM": preprocessor_uploaded_code_script_name
                }
        },
        {
            'Image': pca_image_uri,
            'ModelDataUrl': pca_model_data,
            'Environment': {
                    "TRANSFORM_MODE": "inverse-label-transform"                
            }
        },
        
        {
            'Image': xgb_image_uri,
            'ModelDataUrl': xgb_model_data,
            "Environment": {}
        },
        {
            'Image': preprocessor_image_name,
            'ModelDataUrl': preprocessor_model_data,
            'Environment': {
                    "SAGEMAKER_SUBMIT_DIRECTORY": preprocessor_uploaded_code_s3_prefix,
                    "TRANSFORM_MODE": "inverse-label-transform",
                    "SAGEMAKER_PROGRAM": preprocessor_uploaded_code_script_name
                
                }
        },
    ],
    ExecutionRoleArn = role,
)


In [12]:
inference_pipeline_model_name = model_name

In [13]:
print("inference_pipeline_model_name: \n", inference_pipeline_model_name)

inference_pipeline_model_name: 
 churn-inference-pipeline-2020-08-27-09-30-25


In [14]:
%store inference_pipeline_model_name

Stored 'inference_pipeline_model_name' (str)
